### Модель классификации с использованием **наивного байесовского алгоритма** с анализом данных о заработной плате

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

import warnings
warnings.filterwarnings('ignore') 

In [ ]:
salary_train = pd.read_csv('SalaryData_Train.csv')
salary_train 

In [ ]:
salary_test = pd.read_csv('SalaryData_Test.csv')
salary_test 

Традиционно мы выполняем EDA (разведочный анализ данных), визуализацию, а затем разделяем данные на обучающую и тестовую выборки. Однако у нас уже есть два файла с обучающим и тестовым наборами. Поэтому мы объединим их, выполним EDA и визуализацию, а затем снова разделим данные, чтобы применить алгоритмы машинного обучения.

In [ ]:
#Объединим обучающий и тестовый наборы данных
salary = salary_train._append(salary_test)
salary.reset_index(inplace=True)
salary = salary.drop(['index'], axis=1) 
salary.tail()

# Описательная статистика

In [ ]:
salary.info()

In [ ]:
salary.describe()

In [ ]:
salary.shape

In [ ]:
# Проверка на отсутствующие значения
salary.isna().sum().sum()

In [ ]:
# Числовые признаки

numeric_features = [feature for feature in salary.columns if salary[feature].dtypes != 'O']
print('Числовые признаки:', numeric_features, '\n')

In [ ]:
#В числовых признаках есть 2 типа: дискретные и непрерывные.

discrete_features = [feature for feature in numeric_features if len(salary[feature].unique())<25]
print('Дискретные:', discrete_features, '\n')

continuous_features = [feature for feature in numeric_features if feature not in discrete_features]
print('Непрерывные:', continuous_features, '\n')

In [ ]:
#категориальные признаки

categorical_features = [feature for feature in salary.columns if salary[feature].dtypes == 'O']
print('Категориальные:', categorical_features) 

In [ ]:
# подсчет частоты категориальных переменных

for feature in categorical_features:
    print(salary[feature].value_counts())

In [ ]:
for feature in categorical_features:
    print(feature, 'имеет ', len(salary[feature].unique()), 'меток')

# Разведочный анализ данных

### Обнаружение выбросов

In [ ]:
# для непрерывных признаков -- Boxplot
fig, axes=plt.subplots(4,1,figsize=(14,8))

sns.boxplot(salary.age,  ax=axes[0])
sns.boxplot(salary.capitalgain, ax=axes[1])
sns.boxplot(salary.capitalloss,  ax=axes[2])
sns.boxplot(salary.hoursperweek,  ax=axes[3])

В данных присутствует много выбросов, но мы не можем их отбросить, поскольку они присутствуют в очень большом количестве и могут быть важны для построения модели.

In [ ]:
# для категориальных признаков -- barplot
fig, axes=plt.subplots(len(categorical_features),1,figsize=(14,50))

for i in range (0, len(categorical_features)):
    sns.countplot(salary[categorical_features[i]],ax=axes[i], order=salary[categorical_features[i]].value_counts().index)

## Одномерный анализ

In [19]:
salary_more_than_50=pd.DataFrame()
salary_less_than_50=pd.DataFrame()

salary_more_than_50 = salary.loc[salary['Salary']==' >50K']

salary_less_than_50 = salary.loc[salary['Salary']==' <=50K']

In [ ]:
salary.shape

In [ ]:
(salary_less_than_50.shape[0] + salary_more_than_50.shape[0]) ==salary.shape[0]

### Возрастное распределение по зарплате

In [ ]:
plt.figure(figsize=(15,14))

plt.subplot(3,1,1)
plt.hist(salary_more_than_50.age, bins=[10, 20, 30, 40, 50, 60, 70, 80, 90])
plt.title('Возраст тех, у кого зарплата_больше_50 тыс.')

plt.subplot(3,1,2)
plt.hist(salary_less_than_50.age, color = "orange", bins=[10, 20, 30, 40, 50, 60, 70, 80, 90])
plt.title('Возраст тех, у кого зарплата_меньше_50 тыс.')


age_describe = pd.DataFrame()

age_describe['Возраст тех, у кого зарплата_больше_50 тыс.'] = salary_more_than_50['age'].describe()
age_describe['Возраст тех, у кого зарплата_меньше_50 тыс.'] = salary_less_than_50['age'].describe()

age_describe  = age_describe.drop(['count', '25%', '50%', '75%'])
plt.subplot(3,1,2)
age_describe.plot.bar()

plt.show() 
 

**Вывод:**

- Возраст тех, у кого зарплата больше 50 тыс.: большинство людей находятся в возрастной группе от 30 до 50 лет, и нет никого моложе 20 лет

- Возраст тех, у кого зарплата меньше 50 тыс.: с увеличением возраста количество людей не уменьшается.

### Распределение количества часов в неделю относительно заработной платы

In [ ]:
plt.figure(figsize=(15,14))

plt.subplot(3,1,1)
plt.hist(salary_more_than_50.hoursperweek)
plt.title('Часы/неделя для тех, у кого зарплата больше 50 тыс.')

plt.subplot(3,1,2)
plt.hist(salary_less_than_50.hoursperweek, color = "orange")
plt.title('Часы/неделя для тех, у кого зарплата_меньше_50 тыс.')


Hoursweek_describe = pd.DataFrame()

Hoursweek_describe['Часы/неделя для тех, у кого зарплата_больше_50 тыс.'] = salary_more_than_50['hoursperweek'].describe()
Hoursweek_describe['Часы/неделя для тех, у кого зарплата_меньше_50 тыс.'] = salary_less_than_50['hoursperweek'].describe()

Hoursweek_describe = Hoursweek_describe.drop(['count', '25%', '50%', '75%'])
plt.subplot(3,1,2)
Hoursweek_describe.plot.bar()

plt.show() 
 

**Вывод:**

- В обеих группах большинство работает по 30–40 часов в неделю, а люди, работающие по 80–90 часов, присутствуют в обеих группах.

## Распределение прироста капитала относительно заработной платы

In [ ]:
plt.figure(figsize=(15,14))

plt.subplot(3,1,1)
plt.hist(salary_more_than_50.capitalgain)
plt.title('Прирост капитала у тех, у кого зарплата больше 50 тыс.')

plt.subplot(3,1,2)
plt.hist(salary_less_than_50.capitalgain, color = "orange")
plt.title('Прирост капитала у тех, у кого зарплата меньше 50 тыс.')


capitalgain_describe = pd.DataFrame()

capitalgain_describe['Прирост капитала у тех, у кого зарплата больше 50 тыс.'] = salary_more_than_50['capitalgain'].describe()
capitalgain_describe['Прирост капитала у тех, у кого зарплата меньше 50 тыс.'] = salary_less_than_50['capitalgain'].describe()

capitalgain_describe = capitalgain_describe.drop(['count', '25%', '50%', '75%'])
plt.subplot(3,1,2)
capitalgain_describe.plot.bar()

plt.show() 
 

**Вывод**
- Зарплата более 50 тыс.: прирост капитала для большинства составляет от 0 до 1000

- Зарплата менее 50 тыс.: прирост капитала для большинства составляет менее 500

## Распределение убытков капитала по отношению к зарплате

In [ ]:
plt.figure(figsize=(15,14))

plt.subplot(3,1,1)
plt.hist(salary_more_than_50.capitalloss)
plt.title('Потеря капитала у тех, у кого зарплата больше 50 тыс.')

plt.subplot(3,1,2)
plt.hist(salary_less_than_50.capitalloss, color = "orange")
plt.title('Потеря капитала у тех, у кого зарплата меньше 50 тыс.')


capitalloss_describe = pd.DataFrame()

capitalloss_describe['Потеря капитала у тех, у кого зарплата больше 50 тыс.'] = salary_more_than_50['capitalloss'].describe()
capitalloss_describe['Потеря капитала у тех, у кого зарплата меньше 50 тыс.'] = salary_less_than_50['capitalloss'].describe()

capitalloss_describe = capitalloss_describe.drop(['count', '25%', '50%', '75%'])
plt.subplot(3,1,2)
capitalloss_describe.plot.bar()

plt.show() 
 

## Категориальные переменные относительно зарплаты

In [ ]:
j = 1
categorical_features.remove('Salary') #удаляем столбец зарплата, так как он является целевой переменной.

for feature in categorical_features:
    df = pd.DataFrame()
    df['More than 50k'] = salary[salary['Salary'] == ' >50K'][feature].value_counts()
    df['Less than 50k'] = salary[salary['Salary'] == ' <=50K'][feature].value_counts()
    
    df.plot.bar(figsize=(14,6))
    
    j = j + 1 
    
  

**Вывод:**

- Большинство из обеих групп заняты в частном секторе

- По мере снижения образовательной квалификации число людей в группе «зарплата более 50 тыс.» уменьшается.

- Большинство людей с зарплатой более 50 тыс. состоят в браке.

- Большинство людей с зарплатой менее 50 тыс. - ремонт, транспорт, рыболовство, уборка и другие услуги.

- В то время как люди с зарплатой более 50 тыс. - менеджеры, преподаватели, продавцы.

- Незначительное количество неместных граждан имеют зарплату более 50 тыс.

# Предварительная обработка данных

In [27]:
salary1 = salary.copy()

In [28]:
salary1.drop('Salary',axis=1, inplace =True)

In [ ]:
salary1

In [ ]:
salary1 = pd.get_dummies(salary1)
salary1.head() 

In [ ]:
salary1['Salary'] = salary.Salary
salary1.head() 

In [32]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()
encoder.fit(salary1.Salary)

In [ ]:
salary1.Salary = encoder.transform(salary1.Salary)
salary1

## Разделение данных на обучающую и тестовую выборки

In [ ]:
salary_train.shape

In [ ]:
salary_test.shape

In [37]:
# мы объединили обучающий и тестовый наборы для EDA и визуализации.
# теперь давайте разделим их: первые 30161 - обучающий набор, а остальные 15060 записей — это тестовый набор.

train_set = salary1.iloc[:30161, :]
test_set = salary1.iloc[30161:, :]  

In [ ]:
print("Размер новых данных - {} , {}".format(train_set.shape, test_set.shape))

In [ ]:
train_set 

In [ ]:
x_train = train_set.iloc[:,:102]
x_train 

In [ ]:
y_train = train_set.loc[:,['Salary']]
y_train

In [ ]:
test_set 

In [ ]:
x_test = test_set.iloc[:,:102]
x_test 

In [ ]:
y_test = test_set.loc[:,['Salary']]
y_test

# Наивный байесовский классификатор

In [45]:
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB

from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report as report 

### Gaussian Naïve Bayes

In [ ]:
model_GNB = GaussianNB()

model_GNB.fit(x_train, y_train)

y_pred_GNB = model_GNB.predict(x_test)

#Точность
#первый аргумент — истинные значения, второй аргумент — прогнозируемые значения
print('GaussianNB Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_GNB)))

### Multinomial Naïve Bayes

In [ ]:
model_MNB = MultinomialNB()

model_MNB.fit(x_train, y_train)

y_pred_MNB = model_MNB.predict(x_test)

print('MultinomialNB Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_MNB)))

### Bernoulli Naïve Bayes

In [ ]:
model_BNB = BernoulliNB()

model_BNB.fit(x_train, y_train)

y_pred_BNB = model_BNB.predict(x_test)

print('BernoulliNB Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_BNB)))

### Categorical Naïve Bayes


In [ ]:
model_CNB = CategoricalNB()

model_CNB.fit(x_train, y_train)

y_pred_CNB = model_CNB.predict(x_test)

print('Categorical Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_CNB)))

In [ ]:
print('GaussianNB Model','\n', report(y_test, y_pred_GNB))
print('MultinomialNB Model','\n',report(y_test, y_pred_MNB))
print('BernoulliNB Model','\n',report(y_test, y_pred_BNB))
print('CategoricalNB Model','\n',report(y_test, y_pred_CNB))

... Naïve Bayes показал лучшую точность. Следовательно, мы собираемся продолжать использовать ... для дальнейших метрик.

## Матрица ошибок

In [ ]:
from sklearn.metrics import confusion_matrix as cm 

y_pred_CNB = model_CNB.predict(x_test)
print(cm(y_test,y_pred_CNB)) 

In [ ]:
cm_df=cm(y_test, y_pred_CNB)
class_label = ["No", "Yes"]

df_cm = pd.DataFrame(cm_df, index = class_label, columns = class_label)
sns.heatmap(df_cm, annot = True, fmt = "d")
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("Actual Label")
plt.show()

## ROC AUC

In [ ]:

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr, tpr, thresholds = roc_curve(y_test, model_CNB.predict_proba (x_test)[:,1])
auc = roc_auc_score(y_test, y_pred_CNB)

plt.plot(fpr, tpr, color='red', label='logit model ( area  = %0.2f)' %auc)
plt.plot([0, 1], [0, 1], 'k--')

plt.title('ROC кривая для классификатора')
plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
plt.ylabel('True Positive Rate')
plt.show()

print('AUC-ROC Score:',auc) 

ROC AUC — это сводная информация о производительности классификатора, состоящая из одного числа. Чем выше значение, тем лучше классификатор. Идеальный классификатор будет иметь ROC AUC, равный 1.

## Проверка на переобучение и недообучение

In [ ]:
print('Training set score: {:.2f}%'.format(model_CNB.score(x_train, y_train)*100))

print('Test set score: {:.2f}%'.format(model_CNB.score(x_test, y_test)*100))

Эти два значения должны быть сопоставимы. Таким образом, нет никаких признаков переобучения.